In [20]:
from Util.Counting_and_Detection import findRoot
import cv2
import torch
from Model.pose_higher_hrnet import PoseHigherResolutionNet as get_model_HR
from Detection_Torch.input_param import cfg
from Detection_Torch.input_param import args as args
from glob import glob
import os
import json
import numpy as np
from Util.Counting_and_Detection import predict_strand_num, computeSearchDist

In [21]:
rootDetectionModel = get_model_HR(cfg).cuda()
weights_dir = args.weights_dir_HRNet
rootDetectionModel.load_state_dict(torch.load(weights_dir), strict=False) 

<All keys matched successfully>

In [22]:
IMG_FORMAT = ".jpg"
IMG_FOLDER = "data"

AUGMENT_DATA = True

HEIGHT=480
WIDTH=640

In [23]:
def nms(cur_img_dot_list):
    shape=[480,640]
    cur_target_pt = np.array(cur_img_dot_list,dtype="float32")
    sizeY, sizeX = shape
    _, cur_target_pt, _, _, _ = computeSearchDist(cur_target_pt, sizeY, sizeX)
    cur_target_pt = cur_target_pt[:, :2]
    cur_img_dot_list = cur_target_pt.astype(int).tolist()
    return cur_img_dot_list

In [24]:
dir_list = glob(os.path.join(IMG_FOLDER,"*"))

In [25]:
dir_list

['data\\1', 'data\\11', 'data\\12']

In [27]:
for series_path in dir_list:
    series_id = os.path.basename(series_path)  
    img_path_list = glob(os.path.join(series_path,"*"+IMG_FORMAT))
    img_path_list = sorted(img_path_list)
    for i, img_path in enumerate(img_path_list):
        
        if AUGMENT_DATA:
            if(i%2==1 and i!=len(img_path_list)-1):
                continue
            
        dot_json_name = os.path.basename(img_path).replace(IMG_FORMAT, ".json")
        dot_json_path = os.path.join(os.getcwd(),IMG_FOLDER, series_id, dot_json_name)
        img = cv2.imread(img_path)
        dot_list = nms(findRoot(img, rootDetectionModel, ADDITIONAL_IMG = False))
        
        shape_arr = []
        for point in dot_list:
            shape_dict = {
                "label": "-1",
                "points": [[int(point[0]),int(point[1])]],
                "group_id": None,
                "shape_type": "point",
                "flags": {}
            }
            shape_arr.append(shape_dict)
        
        result_dict = {
            "version": "4.5.9",
            "flags": {},
            "shapes": shape_arr,
            "imagePath": os.path.basename(img_path),
            "imageData": None,
            "imageHeight": HEIGHT,
            "imageWidth": WIDTH
        }
        
        if(os.path.exists(dot_json_path)):
            continue
        else:
            with open(dot_json_path, 'w') as f:
                json.dump(result_dict, f, indent=4)